In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

- association analysis with 연관규칙분석
    - 간단한 연관 룰 생성 {A} -> {B}
    
    - 두 개의 아이템 연관분석을 통해서 
    
    
 <br>
 
 - Python Generators
 
   - iterable sequence of items
   - yields -> next method

Get all possible item pairs for a given order

eg:  order 1:  apple, egg, milk   -->  item pairs: {apple, egg}, {apple, milk}, {egg, milk}
     order 2:  egg, milk          -->  item pairs: {egg, milk}
Count the number of times each item pair appears

eg: {apple, egg}: 1
    {apple, milk}: 1
    {egg, milk}: 2
Here's the generator that implements the above tasks:

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile # zip 파일 풀기

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import os
files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        files.append(filename.split('.zip')[0])
        with zipfile.ZipFile(os.path.join(dirname, filename),"r") as z:
            z.extractall()

In [ ]:
files

In [ ]:
#룰 예시
import numpy as np
from itertools import combinations, groupby
from collections import Counter

# Sample data
orders = np.array([[1,'apple'], [1,'egg'], [1,'milk'], [2,'egg'], [2,'milk']], dtype=object)

# Generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    
    # For each order, generate a list of items in that order
    for order_id, order_object in groupby(orders, lambda x: x[0]):
        item_list = [item[1] for item in order_object]      
    
        # For each item list, generate item pairs, one at a time
        for item_pair in combinations(item_list, 2):
            yield item_pair                                      


# Counter iterates through the item pairs returned by our generator and keeps a tally of their occurrence
Counter(get_item_pairs(orders))

get_item_pairs()

- 아이템 리스트에서 순서에 맞게 item pairs를 생성한다.



![image.png](attachment:7f485bd4-2928-47f4-a63c-d209d8849a83.png)

![image.png](attachment:197449fc-1bbc-435e-abe5-4030b3e88176.png)

![image.png](attachment:f35783b4-8e98-4e5f-9bea-b82c62c0f2de.png)

![image.png](attachment:cca54bc4-4a14-4722-889a-863bb10845a6.png)

In [ ]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display

In [ ]:
# Function that returns the size of an object in MB
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

## Input dataset(order_products_prior)

In [ ]:
departments = pd.read_csv('./'+files[0], dtype = {'departments_id': np.int32})
order_products_train = pd.read_csv('./'+files[2], dtype = {'order_id' : np.int32, 'product_id' : np.int32,
                                                           'add_to_cart_order' : np.int16, 'reordered' : np.int16})
orders = pd.read_csv('./'+files[3], dtype = {'order_id' : np.int32, 'product_id' : np.int32,
                                                           'add_to_cart_order' : np.int16, 'reordered' : np.int16})
orders1 = pd.read_csv('./'+files[4], dtype = {'order_id' : np.int32,'user_id' : np.int32,
                                             'order_number' : np.int16,'order_dow' : np.int16,
                                             'order_hour_of_day' : np.int16})
products = pd.read_csv('./'+files[5],dtype={'products_id': np.int32})
aisles = pd.read_csv('./'+files[6], dtype={'aisles_id': np.int32})

In [ ]:
print('orders -- dimensions: {0};   size: {1}'.format(orders.shape, size(orders)))
display(orders.head())

**association rules 함수에 맞게 order_products_prior형식 변환**

In [ ]:
#df -> series 변환, order_id as index이고 item_id는 값
orders = orders.set_index('order_id')['product_id'].rename('item_id')
display(orders.head(10))
type(orders)

In [ ]:
orders = orders.squeeze() # <- Converts to a pandas series


In [ ]:
type(orders)

**order data를 위한 통계적 요약**

In [ ]:
print('dimensions: {0};   size: {1};   unique_orders: {2};   unique_items: {3}'
      .format(orders.shape, size(orders), len(orders.index.unique()), len(orders.value_counts())))

## Association Rules Function

**Helper functions to the main association rules function**

In [ ]:
# Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))


# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):

    order_item = order_item.reset_index().values
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            

# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]               

**Association rules fuction**

In [ ]:
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Filter from order_item items below min support 
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size= freq(order_item.index)
    qualifying_orders= order_size[order_size >= 2].index
    order_item = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

**Association Rules Mining**

In [ ]:
%%time
rules = association_rules(orders, 0.01)  

In [ ]:
# Replace item ID with item name and display association rules
item_name   = pd.read_csv('./'+files[5],dtype={'products_id': np.int32})
item_name   = item_name.rename(columns={'product_id':'item_id', 'product_name':'item_name'})
rules_final = merge_item_name(rules, item_name).sort_values('lift', ascending=False)
display(rules_final)

![image.png](attachment:6923dc6b-e75d-4bb3-8152-c1ee4476b8a4.png)